## Machine Learning


### Apresentação do Dataset

In [1]:
import pandas as pd
import numpy as np
# execute a linha abaixo apenas se não tiver o Sweetviz
# ! pip install sweetviz


#### Link para Download do Dataset

In [3]:
df = pd.read_excel('saidamerged.xlsx')
df.head()

,Unnamed: 0,Unnamed: 0.1,Título do Ajuste - conforme relatório SUMÁRIO-001,Descrição do Ajuste,Descrição de Impacto,Impacto Principal,Observação,Chave,TIPO,NOME
0,0,1,Encargo Liquidação MCP - Original - Principal (*),Inadimplência do MCP (Mercado de Curto Prazo) ...,São impactados os Agentes inadimplentes da liq...,-,-,Encargo Liquidação MCP - Original - Principal,Não considerar,Encargo Liquidação MCP - Original - Principal
1,1,2,Encargo Liquidação MCP - Estorno - Principal (*),Estorno da Inadimplência do MCP (Mercado de Cu...,Estorno da inadimplência dos agentes CELESC DI...,-,-,Encargo Liquidação MCP - Estorno - Principal,Não considerar,Encargo Liquidação MCP - Estorno - Principal
2,2,3,Encargo Liquidação MCP - Original - Juros (*),Lançamento dos Juros sobre a Inadimplência do ...,São impactados os Agentes inadimplentes da liq...,-,-,Encargo Liquidação MCP - Original - Juros,Não considerar,Encargo Liquidação MCP - Original - Juros
3,3,4,Encargo Liquidação MCP - Estorno - Juros (*),Estorno dos juros sobre a inadimplência do MCP...,Este ajuste impacta a crédito os agentes que p...,-,-,Encargo Liquidação MCP - Estorno - Juros,Não considerar,Encargo Liquidação MCP - Estorno - Juros
4,4,5,Encargo Liquidação MCP - Original - Atualizaçã...,Lançamento da atualização monetária sobre a In...,São impactados os Agentes inadimplentes da liq...,-,-,Encargo Liquidação MCP - Original - Atualizaçã...,Não considerar,Encargo Liquidação MCP - Original - Atualizaçã...


In [2]:
df = pd.read_excel('saidamerged.xlsx', usecols=['Título do Ajuste - conforme relatório SUMÁRIO-001',
       'Descrição do Ajuste', 'Descrição de Impacto', 'Impacto Principal',
       'Observação', 'splitedajuste', 'NOME', 'TIPO', 'Justificativa'])
df.head()

ValueError: Usecols do not match columns, columns expected but not found: ['Justificativa', 'splitedajuste']

In [3]:
df.columns

Index(['Título do Ajuste - conforme relatório SUMÁRIO-001',
       'Descrição do Ajuste', 'Descrição de Impacto', 'Impacto Principal',
       'Observação', 'splitedajuste', 'NOME', 'TIPO', 'Justificativa'],
      dtype='object')

In [4]:
# importing sweetviz
import sweetviz as sv
#analyzing the dataset
advert_report = sv.analyze(df)
#display the report
advert_report.show_html('Advertising.html')


Report Advertising.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.


In [5]:
df.TIPO.value_counts()

ENERGIA           663
Sem categoria     571
ESS               325
Não considerar    211
Name: TIPO, dtype: int64

In [6]:
df['Impacto Principal'].unique()

array(['-', 'Todos os impactos decorrentes de uma recontabilização',
       'Impacto no MRE',
       'Nível de contratação (CCEARs) das distribuidoras. ',
       'Nível de contratação (CCEARs) das distribuidoras ',
       'Nivél de contratação (CCEARs) das distribuidoras. ', nan,
       'Pagamento de encargos por segurança energética.',
       'Pagamento de encargos', 'Encargos',
       'Todos os impactos decorrentes de uma recontabilização .',
       'Perdas de rede básica.', 'Contratação por disponibilidade.',
       'Pagamento de encargos.', 'Encargos.',
       'Rateio para pagamento de encargo por restrição de operação quando PLD<INC<=CMO.',
       'Nível de contratação (CCEN) das distribuidoras ',
       'Impacto no RRH',
       'Nivel de contratação (CCEARs) das distribuidoras ',
       'Balanço Energético e MRE', 'Balanço Energético',
       'Nivél de contratação (CCEARs) das distribuidoras ',
       'Pagamento de encargos por segurança energética',
       'Impacto Encargos'], d

In [8]:
# Bag of Words no sklearn: usamos CountVectorizer
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

In [30]:
text = df['Título do Ajuste - conforme relatório SUMÁRIO-001'].astype(str)

In [31]:
# lista de documentos
text = df['Título do Ajuste - conforme relatório SUMÁRIO-001']
# instancia o transform CountVectorizer
vectorizer = CountVectorizer()
# tokeniza e cria o vocabulário
vectorizer.fit(text)
# mostra o vocabulário criado
print('Vocabulário: ')
print(vectorizer.vocabulary_)
# converte em números
print('\ndimensões da matrix: ')
vector = vectorizer.transform(text)
# # mostra as dimensões da matrix de frequência
print(vector.shape)
# # mostra os documentos codificados
# print(vector.toarray())
# mostra os documentos codificados
values = vectorizer.transform(text)
feature_names = vectorizer.get_feature_names()
print('\n',text[0],'\n',text[1],'\n',text[2],'\n',text[3],'\n',text[4],'\n')
pd.DataFrame(values.toarray(), columns = feature_names)

ValueError: np.nan is an invalid document, expected byte or unicode string.

In [33]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [34]:
# instancia o transform TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer()
# tokeniza e cria o vocabulário
tfidf_vectorizer.fit(text)
# mostra o vocabulário criado
print('Vocabulário: ')
print(tfidf_vectorizer.vocabulary_)
print('\nPeso de cada palavra do vocabulário: ')
print(tfidf_vectorizer.idf_)
# encode document
tfidf_vector = tfidf_vectorizer.transform(text)
# mostra as dimensões da matrix de frequência
print('\nDimensões da matrix: ')
print(tfidf_vector.shape)
# mostra os documentos codificados
val = tfidf_vectorizer.fit_transform(text)
feature_names = tfidf_vectorizer.get_feature_names()
print('\n',text[0],'\n',text[1],'\n',text[2],'\n',text[3],'\n',text[4],'\n')
pd.DataFrame(val.toarray(), columns = feature_names)
# print(tfidf_vector.toarray())

ValueError: np.nan is an invalid document, expected byte or unicode string.